In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 
from qiskit.quantum_info import Statevector
import numpy as np

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.

def random():
    q = QuantumCircuit(1) 
    theta = 2 * np.arccos(1/np.sqrt(3))
    q.ry(theta,0)
    q.measure_all() 
    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)

    bit = list(counts.keys())[0]
    
    if  bit == '0':
        return '1'
    else:
        q2 = QuantumCircuit(1)
        q2.h(0)
        q2.measure_all() 
        backend = BasicSimulator()
        compiled = transpile(q2, backend)
        job_sim = backend.run(compiled, shots=1)
        result_sim = job_sim.result() 
        counts = result_sim.get_counts(compiled)
        bit2 = list(counts.keys())[0] 
        if bit2 == '0':
            return '2'
        else:
            return '3'

def randomList(n):
    bs = []
    while len(bs) < (n):
        bit = str(random())
        bs.append(bit)
    return bs 

def entangledPair():
    q = QuantumCircuit(2)
    q.x(1)
    q.h(0)
    q.z(0)
    q.cx(0,1)
    return q

N = 20 
AOperators = randomList((9*N)//2)
# 1 = X
# 2 = W
# 3 = Z
import collections
counts = collections.Counter(AOperators)
print(counts)
print("Alice operators:   %s" % AOperators) 

BOperators = randomList((9*N)//2)
# 1 = W
# 2 = Z
# 3 = V
print("Bob operators:   %s" % BOperators) 

        
BBits = []
ABits = []

root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 


B0_transform_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ] ]

B1_transform_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ] ]

for i in range ((9*N)//2):
    A = AOperators[i]
    B = BOperators[i]
    circuit = entangledPair()
    
    # transform Alice's qubit
    if (A == '1'):
        circuit.x(0)
    elif (A =='2'):
        circuit.unitary(B0_transform_matrix,[0])
    else:
        circuit.z(0)
        
    # transform Bob's qubit
    if (B == '1'):
        circuit.unitary(B0_transform_matrix,[1])
    elif (B =='2'):
        circuit.z(1)
    else:
        circuit.unitary(B1_transform_matrix,[1])

    circuit.measure_all()
    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result()
    
    counts = result_sim.get_counts(compiled)
    
    bit = int(list(counts.keys())[0][1]) 
    Abit = int(list(counts.keys())[0][0]) 
    BBits = BBits + [bit] 
    ABits = ABits + [Abit]


sharedKey = [] 
for i in range ((9*N)//2):
    if ((AOperators[i] == '2' and BOperators[i] == '1') or
        (AOperators[i] == '3' and BOperators[i] == '2' )):

        if ABits[i] == BBits[i]:
            print("ERROR mismatch at", i)
            #print("Alice", AOperators[i],ABits[i])
            #print("Bob", BOperators[i],BBits[i])
        else:
            sharedKey = sharedKey + [ABits[i]] 

print("Shared key (length %i): %s" % (len(sharedKey), sharedKey)) 


Counter({'1': 34, '3': 29, '2': 27})
Alice operators:   ['1', '3', '1', '3', '1', '3', '3', '2', '3', '2', '2', '2', '3', '3', '3', '3', '2', '3', '3', '1', '1', '1', '1', '2', '2', '1', '1', '2', '3', '3', '2', '1', '1', '1', '2', '1', '2', '2', '3', '3', '2', '1', '3', '2', '1', '2', '2', '2', '2', '3', '2', '1', '3', '2', '2', '1', '2', '2', '1', '3', '1', '3', '3', '2', '1', '1', '1', '3', '1', '1', '3', '1', '1', '3', '1', '1', '3', '1', '2', '2', '3', '1', '1', '1', '3', '2', '1', '3', '1', '3']
Bob operators:   ['2', '3', '1', '1', '2', '1', '1', '1', '2', '1', '1', '2', '1', '3', '1', '3', '1', '3', '3', '2', '2', '1', '1', '1', '1', '1', '1', '2', '1', '2', '2', '2', '3', '2', '1', '3', '3', '1', '2', '1', '1', '3', '3', '1', '3', '1', '2', '2', '3', '1', '1', '3', '2', '1', '1', '3', '2', '1', '2', '2', '1', '2', '1', '2', '1', '2', '2', '2', '3', '3', '2', '1', '3', '3', '3', '2', '1', '2', '1', '3', '3', '2', '1', '1', '3', '2', '3', '3', '3', '2']
Shared key (length 25): [